In [ ]:
from google.colab import files
uploaded = files.upload()  # Click "Choose Files" and select your products.csv

Saving combined_stock_features.csv to combined_stock_features.csv


In [ ]:
import pandas as pd
import io

# Read the uploaded file
df = pd.read_csv(io.BytesIO(uploaded['combined_stock_features.csv']))
print(df)

# Or if you just want to save it first:
with open('combined_stock_features.csv', 'wb') as f:
    f.write(uploaded['combined_stock_features.csv'])

df = pd.read_csv('combined_stock_features.csv')
print(df)

             Date       Close        High         Low        Open  \
0      2020-10-15  117.704178  118.181974  115.207927  115.763733   
1      2020-10-16  116.056252  118.523258  115.851482  118.259977   
2      2020-10-19  113.091957  117.421391  112.779926  116.972846   
3      2020-10-20  114.583862  116.017259  112.750672  113.306478   
4      2020-10-21  113.959785  115.753963  113.550238  113.764761   
...           ...         ...         ...         ...         ...   
79081  2023-12-22   97.674698   98.661891   97.588434   98.058068   
79082  2023-12-26   97.895149   98.748161   97.875984   98.470212   
79083  2023-12-27   97.435097   98.288108   97.128389   97.799302   
79084  2023-12-28   96.026192   97.387176   95.968680   97.176318   
79085  2023-12-29   95.824921   96.409563   95.479881   96.198712   

            Volume        MA10        MA50       MA100       MA200        RSI  \
0      112559200.0  114.778874  114.007471  101.501437   86.176184  63.102298   
1      11

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# # Load the enhanced dataset
# def load_enhanced_data(ticker="MSFT"):
#     df = pd.read_csv(f"{ticker}_stock_features.csv")
#     df['Date'] = pd.to_datetime(df['Date'])  # Ensure Date is datetime
#     df.set_index('Date', inplace=True)
#     return df

# ticker = "MSFT"
# data = load_enhanced_data(ticker)
# print(data.head())  # Verify all features are loaded

In [ ]:
ticker_to_id = {ticker: idx for idx, ticker in enumerate(df['Ticker'].unique())}
df['Ticker_ID'] = df['Ticker'].map(ticker_to_id)

In [ ]:
# Select features and target
features = ['Close', 'MA100', 'RSI', 'Norm_Volume', 'Bollinger_Upper', 'Bollinger_Lower', 'Log_Returns', 'Ticker_ID']
data = df[features]  # Keep only selected features

In [ ]:
data

,Close,MA100,RSI,Norm_Volume,Bollinger_Upper,Bollinger_Lower,Log_Returns,Ticker_ID
0,117.704178,101.501437,63.102298,0.630404,121.173481,102.621430,-0.003969,0
1,116.056252,101.887923,56.510599,0.652642,121.189614,103.792967,-0.014100,0
2,113.091957,102.244426,52.833616,0.688337,121.133551,104.424339,-0.025874,0
3,114.583862,102.616603,52.563324,0.717129,121.254079,104.859618,0.013106,0
4,113.959785,102.973024,50.121869,0.524334,120.661635,106.402785,-0.005461,0
...,...,...,...,...,...,...,...,...
79081,97.674698,103.023652,48.316035,0.588664,100.445159,93.591178,0.001768,97
79082,97.895149,102.993496,56.213436,0.776447,100.160121,93.701780,0.002254,97
79083,97.435097,102.955915,59.937659,0.673882,99.815751,93.831459,-0.004711,97
79084,96.026192,102.906317,56.502598,0.758253,99.671195,93.769952,-0.014565,97


In [ ]:
def min_max_scaling(data):
    """Scale data to [0, 1] range and return scaled data + min/max values"""
    min_val = np.min(data)
    max_val = np.max(data)
    # Handle case where all values are identical (avoid division by zero)
    if max_val == min_val:
        return np.zeros_like(data), min_val, max_val
    return (data - min_val) / (max_val - min_val), min_val, max_val

In [ ]:
def scale_features(df):
    """Scale each feature column independently"""
    scaled_data = {}
    scalers = {}  # To store min/max for each feature

    for col in df.columns:
        if col == 'Ticker_ID':
            scaled_data[col] = df[col].values  # Keep as is (integer IDs)
            continue

        scaled_values, min_val, max_val = min_max_scaling(df[col].values)
        if col == 'Log_Returns':
            scalers['target'] = (min_val, max_val)
        else:
            scalers[col] = (min_val, max_val)

        scaled_data[col] = scaled_values

    return pd.DataFrame(scaled_data), scalers

# Scale all features
scaled_data, scalers = scale_features(data)

# Split into train/test (keeping temporal order)
split_ratio = 0.8
split_index = int(len(scaled_data) * split_ratio)
train_data = scaled_data.iloc[:split_index]
test_data = scaled_data.iloc[split_index:]

In [ ]:
print(train_data.columns)
print(test_data.columns)

Index(['Close', 'MA100', 'RSI', 'Norm_Volume', 'Bollinger_Upper',
       'Bollinger_Lower', 'Log_Returns', 'Ticker_ID'],
      dtype='object')
Index(['Close', 'MA100', 'RSI', 'Norm_Volume', 'Bollinger_Upper',
       'Bollinger_Lower', 'Log_Returns', 'Ticker_ID'],
      dtype='object')


In [ ]:
# Create ticker mapping and ticker IDs
unique_tickers = list(set(train_data['Ticker_ID'].values) | set(test_data['Ticker_ID'].values))
ticker_mapping = {ticker: idx for idx, ticker in enumerate(unique_tickers)}

train_ticker_ids = train_data['Ticker_ID'].values
test_ticker_ids = test_data['Ticker_ID'].values

In [ ]:
def create_sequences_multi(data, ticker_ids, seq_length=50, target_col='Log_Returns'):
    """Create sequences from multiple features"""
    X, y, ticker_seq_list = [], [], []
    data_values = data.values  # Convert to numpy array

    for ticker_id in np.unique(ticker_ids):
        ticker_data = data[data['Ticker_ID'] == ticker_id]
        #ticker_data = ticker_data.sort_values('Date')  # or whatever date column you use
        features = ticker_data.drop(['Log_Returns', 'Ticker_ID'], axis=1).values
        targets = ticker_data['Log_Returns'].values

        for i in range(len(features) - seq_length):
            # All features for the sequence
            X.append(data_values[i:i+seq_length, :])
            # Get the sequence of ticker IDs for this batch
            # tickers = ticker_ids[i:i+seq_length]
            ticker_seq_list.append(ticker_id)

            # Only the Close price as target
            y.append(data_values[i+seq_length, data.columns.get_loc(target_col)])

    return np.array(X), np.array(y), np.array(ticker_seq_list)

# Create sequences
seq_length = 50
X_train, y_train, train_tickers = create_sequences_multi(train_data, train_ticker_ids, seq_length)
X_test, y_test, test_tickers = create_sequences_multi(test_data, test_ticker_ids, seq_length)

# Note: No need to reshape since we already have 3D (samples, timesteps, features)
print(f"Training shapes - X: {X_train.shape}, y: {y_train.shape}")
print(f"Test shapes - X: {X_test.shape}, y: {y_test.shape}")

Training shapes - X: (59318, 50, 8), y: (59318,)
Test shapes - X: (14818, 50, 8), y: (14818,)


In [ ]:
# Add Adam optimizer support in LSTM class

class LSTM:
    def __init__(self, input_dim, hidden_dim, output_dim, ticker_dim, embedding_dim, learning_rate=0.01, beta1=0.9, beta2=0.999, epsilon=1e-8):
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.lr = learning_rate
        self.ticker_dim = ticker_dim
        self.embedding_dim = embedding_dim
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.t = 1  # time step for Adam

        # Combined input size: hidden + input + ticker embedding
        concat_dim = hidden_dim + input_dim + embedding_dim

        # Initialize weights
        self.Wf = np.random.randn(hidden_dim, concat_dim) * 0.01
        self.bf = np.zeros((hidden_dim, 1))

        self.Wi = np.random.randn(hidden_dim, concat_dim) * 0.01
        self.bi = np.zeros((hidden_dim, 1))

        self.Wc = np.random.randn(hidden_dim, concat_dim) * 0.01
        self.bc = np.zeros((hidden_dim, 1))

        self.Wo = np.random.randn(hidden_dim, concat_dim) * 0.01
        self.bo = np.zeros((hidden_dim, 1))

        # Output layer
        self.Wy = np.random.randn(output_dim, hidden_dim) * 0.01
        self.by = np.zeros((output_dim, 1))

        # Initialize embedding matrix for Ticker_ID
        self.ticker_embedding = np.random.randn(self.ticker_dim, self.embedding_dim) * 0.01

        # Initialize Adam moment estimates
        self._init_adam_params()

    def _init_adam_params(self):
        self.m = {}
        self.v = {}
        for param_name in ['Wf', 'Wi', 'Wc', 'Wo', 'Wy', 'bf', 'bi', 'bc', 'bo', 'by']:
            param = getattr(self, param_name)
            self.m[param_name] = np.zeros_like(param)
            self.v[param_name] = np.zeros_like(param)

    def get_ticker_embedding(self, ticker_id):
        """Return the embedding for the ticker ID"""
        return self.ticker_embedding[ticker_id].reshape(-1, 1)

    def sigmoid(self, x): return 1 / (1 + np.exp(-x))
    def dsigmoid(self, x): return x * (1 - x)
    def tanh(self, x): return np.tanh(x)
    def dtanh(self, x): return 1 - x ** 2


    def forward(self, x_seq, ticker_id, h=None, c=None):

      if h is None:
          h = np.zeros((self.hidden_dim, 1))
      if c is None:
          c = np.zeros((self.hidden_dim, 1))
      self.caches = []

    # Get embedding once per sequence (not per timestep)
      ticker_embedding = self.get_ticker_embedding(ticker_id).reshape(-1, 1)

      for x in x_seq:
          x = x.reshape(self.input_dim, 1)

          # Concatenate previous hidden state, input, and ticker embedding
          concat = np.vstack((h, x, ticker_embedding))

          ft = self.sigmoid(np.dot(self.Wf, concat) + self.bf)
          it = self.sigmoid(np.dot(self.Wi, concat) + self.bi)
          c_tilde = self.tanh(np.dot(self.Wc, concat) + self.bc)
          c = ft * c + it * c_tilde
          ot = self.sigmoid(np.dot(self.Wo, concat) + self.bo)
          h = ot * self.tanh(c)

          self.caches.append((h, c, ft, it, c_tilde, ot, concat))

      y_hat = np.dot(self.Wy, h) + self.by
      return y_hat, h, c

    def backward(self, x_seq, y_hat, y_true):
        dh_next = np.zeros((self.hidden_dim, 1))
        dc_next = np.zeros((self.hidden_dim, 1))

        grads = {
            'Wf': np.zeros_like(self.Wf), 'Wi': np.zeros_like(self.Wi),
            'Wc': np.zeros_like(self.Wc), 'Wo': np.zeros_like(self.Wo),
            'Wy': np.zeros_like(self.Wy),
            'bf': np.zeros_like(self.bf), 'bi': np.zeros_like(self.bi),
            'bc': np.zeros_like(self.bc), 'bo': np.zeros_like(self.bo),
            'by': np.zeros_like(self.by)
        }

        dy = y_hat - y_true
        grads['Wy'] += np.dot(dy, self.caches[-1][0].T)
        grads['by'] += dy

        dh = np.dot(self.Wy.T, dy) + dh_next

        for t in reversed(range(len(x_seq))):
            h, c, ft, it, c_tilde, ot, concat = self.caches[t]
            c_prev = self.caches[t - 1][1] if t > 0 else np.zeros_like(c)

            do = dh * self.tanh(c)
            do_raw = do * self.dsigmoid(ot)

            dc = dh * ot * self.dtanh(self.tanh(c)) + dc_next
            dc_tilde = dc * it
            dc_tilde_raw = dc_tilde * self.dtanh(c_tilde)

            di = dc * c_tilde
            di_raw = di * self.dsigmoid(it)

            df = dc * c_prev
            df_raw = df * self.dsigmoid(ft)

            grads['Wf'] += np.dot(df_raw, concat.T)
            grads['Wi'] += np.dot(di_raw, concat.T)
            grads['Wc'] += np.dot(dc_tilde_raw, concat.T)
            grads['Wo'] += np.dot(do_raw, concat.T)

            grads['bf'] += df_raw
            grads['bi'] += di_raw
            grads['bc'] += dc_tilde_raw
            grads['bo'] += do_raw

            dconcat = (np.dot(self.Wf.T, df_raw) +
                       np.dot(self.Wi.T, di_raw) +
                       np.dot(self.Wc.T, dc_tilde_raw) +
                       np.dot(self.Wo.T, do_raw))

            dh = dconcat[:self.hidden_dim, :]
            dc_next = dc * ft

        self._apply_adam(grads)
        self.t += 1  # Increment timestep

    def _apply_adam(self, grads):
        for param_name in grads:
            grad = grads[param_name]
            self.m[param_name] = self.beta1 * self.m[param_name] + (1 - self.beta1) * grad
            self.v[param_name] = self.beta2 * self.v[param_name] + (1 - self.beta2) * (grad ** 2)

            m_hat = self.m[param_name] / (1 - self.beta1 ** self.t)
            v_hat = self.v[param_name] / (1 - self.beta2 ** self.t)

            param = getattr(self, param_name)
            param -= self.lr * m_hat / (np.sqrt(v_hat) + self.epsilon)
            setattr(self, param_name, param)

    def train(self, X_train, y_train, ticker_ids_train, epochs=10, batch_size=32):
        for epoch in range(epochs):
            total_loss = 0
            indices = np.arange(len(X_train))
            np.random.shuffle(indices)

            for i in range(0, len(X_train), batch_size):
                batch_indices = indices[i:i+batch_size]
                batch_loss = 0
                batch_grads = []

                for j in batch_indices:
                    x_seq = X_train[j]
                    y_true = y_train[j].reshape(self.output_dim, 1)
                    ticker_ids = ticker_ids_train[j]
                    y_hat, _, _ = self.forward(x_seq, ticker_ids)
                    loss = np.mean((y_hat - y_true) ** 2)
                    batch_loss += loss

                    self.backward(x_seq, y_hat, y_true)

                total_loss += batch_loss

            print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss:.6f}")


In [ ]:
X_train, y_train, train_tickers = create_sequences_multi(train_data, train_data['Ticker_ID'].values, seq_length)
X_test, y_test, test_tickers = create_sequences_multi(test_data, test_data['Ticker_ID'].values, seq_length)

lstm = LSTM(input_dim=X_train.shape[2], hidden_dim=128, output_dim=1,
            ticker_dim=len(ticker_mapping), embedding_dim=32, learning_rate=0.001)

lstm.train(X_train, y_train, train_tickers, epochs=150, batch_size=32)


Epoch 1/150, Loss: 55.385756


In [ ]:
def predict_multi(lstm, X_test, test_tickers, scalers):
    predictions = []

    for i in range(len(X_test)):
        x_seq = X_test[i]
        ticker_ids = test_tickers[i]
        # Reinitialize h and c for stateless predictions
        h_prev = np.zeros((lstm.hidden_dim, 1))
        c_prev = np.zeros((lstm.hidden_dim, 1))
        y_pred, _, _ = lstm.forward(x_seq, ticker_ids, h_prev, c_prev)
        predictions.append(y_pred.flatten()[0])

    # Inverse scale predictions
    min_target, max_target = scalers['target']
    predictions = np.array(predictions) * (max_target - min_target) + min_target
    return predictions


In [ ]:
# Inverse transform y_test
min_target, max_target = scalers['target']
y_test_actual = y_test * (max_target - min_target) + min_target

# Generate predictions
predictions = predict_multi(lstm, X_test, scalers)

# Plotting
plt.figure(figsize=(14, 6))
plt.plot(y_test_actual, label='Actual Price', color='blue')
plt.plot(predictions, label='Predicted Price', color='red')
plt.title(f"{ticker} Stock Price Prediction - Actual vs Predicted")
plt.xlabel("Time Steps")
plt.ylabel("Price")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Align the dates with predictions (considering sequence length offset)
test_dates = data.index[split_index + seq_length:]  # since we lose 'seq_length' rows when creating sequences

# Create a DataFrame with Date, Actual Price, and Predicted Price
results_df = pd.DataFrame({
    'Date': test_dates,
    'Actual Price': y_test_actual,
    'Predicted Price': predictions
})

# Optional: round for cleaner view
results_df['Actual Price'] = results_df['Actual Price'].round(2)
results_df['Predicted Price'] = results_df['Predicted Price'].round(2)

# Display the table
print(results_df.head(10))  # Show first 10 rows
